In [2]:
import pandas as pd
import src.condition as con
import numpy as np

In [39]:
data = pd.read_excel('example_data.xlsx')

In [41]:
data.duration=pd.to_timedelta(data.duration.astype(str))

In [44]:
data

,date,duration,amplitude
0,2018-09-06 00:00:17,00:00:13,-2
1,2018-09-06 00:00:39,00:00:22,3
2,2018-09-06 00:00:48,00:00:09,-1
3,2018-09-06 00:00:49,00:00:01,4
4,2018-09-06 00:00:50,00:00:01,-7
5,2018-09-06 00:00:51,00:00:01,7
6,2018-09-06 00:00:53,00:00:02,-9
7,2018-09-06 00:01:01,00:00:08,7
8,2018-09-06 00:01:04,00:00:03,-11
9,2018-09-06 00:01:05,00:00:01,5


In [46]:
bools_duration = data.duration.dt.seconds<4

In [53]:
bools_amplitude = get_bools(data.amplitude,5,3)

In [54]:
bools=bools_duration & bools_amplitude

In [56]:
bools

0     False
1     False
2     False
3      True
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12     True
13     True
14    False
15    False
16    False
17    False
18    False
19    False
dtype: bool

In [48]:
signs= sign_data(data.amplitude,bools)

In [50]:
data['sign']=signs

In [66]:
data.groupby('sign').agg({'amplitude':'sum','duration':'sum','date':chg.last_time})

,amplitude,duration,date
sign,,,
1,-2,00:00:13,2018-09-06 00:00:17
2,3,00:00:22,2018-09-06 00:00:39
3,-4,00:00:11,2018-09-06 00:00:50
4,7,00:00:01,2018-09-06 00:00:51
5,-9,00:00:02,2018-09-06 00:00:53
6,7,00:00:08,2018-09-06 00:01:01
7,-8,00:00:16,2018-09-06 00:01:17
8,9,00:00:07,2018-09-06 00:01:24
9,-9,00:00:47,2018-09-06 00:02:11


In [64]:
import src.change as chg

In [49]:
def get_bools(data : pd.Series, negative_measure : float, pozitive_measure : float) -> pd.Series:
    bools=data.apply(condition_amp)
    bools[0]=False
    return bools

In [50]:
def condition_amp(x,negative_measure=4,pozitive_measure=6):
    if x <0:
        return abs(x)<=negative_measure
    if x >0:
        return abs(x)<=pozitive_measure

In [51]:
def sign_data(data : pd.Series, bools : pd.Series) -> pd.Series:
    a=data.mask(bools).fillna(method='ffill')
    return np.sign(a).diff().ne(0).cumsum()